# Plausi der Messwerte der Velo- und Fussgaengerzaehlung
Datum: 15.08.2022

**Ausgangslage:**

Das TAZ liefert wöchentlich die Zähldaten auf die OGD-Dropzone. 

Auf OGD sind sie unter folgender URL zu finden: https://data.stadt-zuerich.ch/dataset/ted_taz_verkehrszaehlungen_werte_fussgaenger_velo

Abgeschlossene Jahre sind als einzelne Ressource downloadbar. Das aktuelle Jahr wird laufend nachgeführt.



## Einstellungen
### Importiere die notwendigen Packages

In [1]:
#%pip install openpyxl geopandas altair fiona requests folium mplleaflet contextily seaborn datetime plotly

In [48]:
import numpy as np
import pandas as pd
import geopandas as gpd
import fiona; 
import mplleaflet
import pivottablejs
from pivottablejs import pivot_ui

import altair as alt
import datetime
import folium 
import plotly.express as px

import requests
import io
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100


Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [49]:
pd.options.display.float_format = '{:.1f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

In [50]:
SSL_VERIFY = False
# evtl. SSL_VERIFY auf False setzen wenn die Verbindung zu https://www.gemeinderat-zuerich.ch nicht klappt (z.B. wegen Proxy)
# Um die SSL Verifikation auszustellen, bitte die nächste Zeile einkommentieren ("#" entfernen)
# SSL_VERIFY = False

In [51]:
if not SSL_VERIFY:
    import urllib3
    urllib3.disable_warnings()

### Zeitvariabeln
Bestimme den aktuellst geladenen Monat. Hier ist es der Stand vor 2 Monaten. 
Bestimme noch weitere evt. sinnvolle Zeitvariabeln.

Zum Unterschied zwischen import `datetime` und `from datetime import datetime`, siehe https://stackoverflow.com/questions/15707532/import-datetime-v-s-from-datetime-import-datetime

Zuerst die Zeitvariabeln als Strings

In [52]:
geojson_url = "https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Move_and_Chill?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=view_moveandchill"


In [53]:
    r = requests.get(geojson_url, verify=False)  
    r.encoding = 'utf-8'
    
    gdf_data2betested = gpd.read_file(
        io.StringIO(r.text), convert_dates=True, keep_default_dates=True, lines=True
    )

gdf_data2betested.dtypes
#gdf_data2betested.columns
#gdf_data2betested

humidity        float64
id                int64
latitude        float64
longitude       float64
noise           float64
objectid        float64
sensor_eui       object
sit             float64
temperature     float64
zeitpunkt        object
geometry       geometry
dtype: object

In [54]:
now = datetime.date.today()
date_today = now.strftime("%Y-%m-%d")
year_today = now.strftime("%Y")
month_today = now.strftime("%m")
day_today = now.strftime("%d")
print(now, date_today, year_today, month_today,day_today )

2022-08-26 2022-08-26 2022 08 26


Und hier noch die Zeitvariabeln als Integers:
- `aktuellesJahr`
- `aktuellerMonat`: Der gerade jetzt aktuelle Monat
- `selectedMonat`: Der aktuellste Monat in den Daten. In der Regel zwei Monate her.

In [55]:
int_times = now.timetuple()

aktuellesJahr = int_times[0]
aktuellerMonat = int_times[1]
selectedMonat = int_times[1]-2

print(aktuellesJahr, 
      aktuellerMonat,
      'datenstand: ', 
      selectedMonat,
     int_times)


2022 8 datenstand:  6 time.struct_time(tm_year=2022, tm_mon=8, tm_mday=26, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=4, tm_yday=238, tm_isdst=-1)


## Importiere die Geodaten und Identifikatoren zu den Messdaten

- URL zu den Zählstellenstandorten: https://data.stadt-zuerich.ch/dataset/geo_standorte_der_automatischen_fuss__und_velozaehlungen
- Für den Datenimport dieser Geodaten verwenden wir den GeoJSON-Service unter: 
https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Standorte_der_automatischen_Fuss__und_Velozaehlungen?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=view_eco_standorte

Attribute:

- KORREKTURFAKTOR	Der Faktor mit dem aus den Gerätewerten die effektiven Frequenzen berechnet werden können. Wertebereiche sind zwischen 0 und 2. Typ NUMBER.
- ID1	. Typ NUMBER.
- FK_ZAEHLER	Geräte-ID. Typ VARCHAR2.
- ABKUERZUNG	Abkürzung des Zählstandorts. Typ VARCHAR2.
- BEZEICHNUNG	Bezeichnung des Zählstandorts. Typ VARCHAR2.
- VON	Standort gültig von. Typ DATE.
- BIS	Standort gütlig bis (falls NULL ist der Standort aktuell). Typ DATE.
- RICHTUNG_IN	Richtungsangabe für Feld IN (gilt für Fuss- und Veloverkehr). Typ VARCHAR2.
- RICHTUNG_OUT	Richtungsangabe für Feld OUT (gilt für Fuss- und Veloverkehr). Typ VARCHAR2.
- GEOMETRIE	. Typ SDO_GEOMETRY.

In [56]:
geojson_url = "https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Standorte_der_automatischen_Fuss__und_Velozaehlungen?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=view_eco_standorte"


In [57]:
    r = requests.get(geojson_url, verify=False)  
    r.encoding = 'utf-8'
    
    gdf_zaehler_standorte = gpd.read_file(
        io.StringIO(r.text)
        , convert_dates=True    
        , keep_default_dates=True
        , lines=True
    )

gdf_zaehler_standorte.dtypes
#gdf_zaehler_standorte.columns
#gdf_zaehler_standorte

id                   object
abkuerzung           object
bezeichnung          object
bis                  object
fk_zaehler           object
id1                   int64
korrekturfaktor     float64
objectid            float64
richtung_in          object
richtung_out         object
von                  object
geometry           geometry
dtype: object

In [58]:
gdf_zaehler_standorte=gdf_zaehler_standorte \
    .copy() \
    .assign(
        timestamp_von = lambda x: pd.to_datetime(gdf_zaehler_standorte['von'])
        ,timestamp_bis = lambda x: pd.to_datetime(gdf_zaehler_standorte['bis'])    
    )
gdf_zaehler_standorte.dtypes

id                         object
abkuerzung                 object
bezeichnung                object
bis                        object
fk_zaehler                 object
id1                         int64
korrekturfaktor           float64
objectid                  float64
richtung_in                object
richtung_out               object
von                        object
geometry                 geometry
timestamp_von      datetime64[ns]
timestamp_bis      datetime64[ns]
dtype: object

In [59]:
#gdf_zaehler_standorte.plot()
#gdf_zaehler_standorte.fk_zaehler.unique()

## Importiere die bereits veröffentlichten Zeitreihen der Messstationen 

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

### Setze einige Pfadvariabeln

- Der Packagename ist eigentlich der **Verzeichnisname** unter dem die Daten und Metadaten auf der Dropzone abgelegt werden.
- Definiert wird er bei SASA-Prozessen auf dem **Produkte-Sharepoint ([Link](https://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/PersonalViews.aspx?PageView=Personal&ShowWebPart={6087A3E7-8AC8-40BA-8278-DECFACE124FF}))**.
- Der Packagename wird auf CKAN teil der URL, daher ist die exakte Schreibweise wichtig.

Beachte: im Packagename müssen alle Buchstaben **klein** geschrieben werden. Dies weil CKAN aus grossen kleine Buchstaben macht.

**BITTE HIER ANPASSEN**

In [61]:
package_name = "ted_taz_verkehrszaehlungen_werte_fussgaenger_velo"

Datensatz NUR für das aktuelle Jahr

In [62]:
#Datasetname für das aktuelle Jahr
dataset_name = year_today+"_verkehrszaehlungen_werte_fussgaenger_velo.csv"

**Statische Pfade in DWH-Dropzones**

In [63]:
dropzone_path_integ = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_TAZ"

In [64]:
dropzone_path_prod = r"\\szh\ssz\applikationen\OGD_Dropzone\TAZ"

**Statische Pfade CKAN-URLs**

In [65]:
ckan_integ_url ="https://data.integ.stadt-zuerich.ch/dataset/"

In [66]:
ckan_prod_url ="https://data.stadt-zuerich.ch/dataset/"

### Checke die Metadaten auf der CKAN INTEG- oder PROD-Webseite

Offenbar lassen sich aktuell im Markdownteil keine Variabeln ausführen, daher gehen wir wie unten gezeigt vor. Siehe dazu: https://data-dive.com/jupyterlab-markdown-cells-include-variables
Instead of setting the cell to Markdown, create Markdown from withnin a code cell! We can just use python variable replacement syntax to make the text dynamic

**Dataset auf INTEG Datenkatalog**:   https://data.integ.stadt-zuerich.ch/dataset/ted_taz_verkehrszaehlungen_werte_fussgaenger_velo/
download/2022_verkehrszaehlungen_werte_fussgaenger_velo.csv

**Dataset auf PROD Datenkatalog**:  https://data.stadt-zuerich.ch/dataset/ted_taz_verkehrszaehlungen_werte_fussgaenger_velo/
download/2022_verkehrszaehlungen_werte_fussgaenger_velo.csv

In [67]:
#Die Datasets sind nur zum Testen auf INT-DWH-Dropzone. Wenn der Test vorbei ist, sind sie auf PROD. 
# Über den Status kann man einfach switchen

status = "prod"; #prod vs something else
data_source = "web"; #dropzone vs something else
print(status+" - "+ data_source)

prod - web


In [68]:
# Filepath
if status == "prod":
    if data_source == "dropzone":
            fp = dropzone_path_prod+"\\"+ package_name +"\\"+dataset_name
            print("fp lautet:"+fp)
    else:
        #fp = r"https://data.stadt-zuerich.ch/dataset/bau_neubau_whg_bausm_rinh_geb_projstatus_quartier_seit2009_od5011/download/BAU501OD5011.csv"
        fp = ckan_prod_url+package_name+'/download/'+dataset_name
        print("fp lautet:"+fp)
else:
    if data_source == "dropzone":
        fp = dropzone_path_integ+"\\"+ package_name +"\\"+dataset_name
        print("fp lautet:"+fp)
    else:
        #fp = r"https://data.stadt-zuerich.ch/dataset/bau_neubau_whg_bausm_rinh_geb_projstatus_quartier_seit2009_od5011/download/BAU501OD5011.csv"
        fp = ckan_integ_url+package_name+'/download/'+dataset_name
        print("fp lautet:"+fp)


fp lautet:https://data.stadt-zuerich.ch/dataset/ted_taz_verkehrszaehlungen_werte_fussgaenger_velo/download/2022_verkehrszaehlungen_werte_fussgaenger_velo.csv


Beachte, wie das SAS Datum (ohne Format) in ein UNIX Datum umgerechnet und als Datumsformat dargestellt wird! Siehe dazu `https://stackoverflow.com/questions/26923564/convert-sas-numeric-to-python-datetime`

In [69]:
# Read the data
if data_source == "dropzone":
    data2betested = pd.read_csv(
        fp
        , sep=','
        ,parse_dates=['DATUM']
        ,low_memory=False
    )
    print("dropzone")
else:
    r = requests.get(fp, verify=False)  
    r.encoding = 'utf-8'
    df_data2betested = pd.read_csv(
        io.StringIO(r.text)
        ,parse_dates=['DATUM']
        # KONVERTIERE DAS SAS DATUM IN EIN UNIXDATUM UND FORMATIERE ES
        #, date_parser=lambda s: epoch + datetime.timedelta(days=int(s))
        ,low_memory=False)
    print("web")

df_data2betested.dtypes

web


FK_ZAEHLER             object
FK_STANDORT             int64
DATUM          datetime64[ns]
VELO_IN               float64
VELO_OUT              float64
FUSS_IN               float64
FUSS_OUT              float64
OST                     int64
NORD                    int64
dtype: object

Ich importiere die Datumsfelder bewussts als STRINGS. Nicht im Datumsformat.
Dies erlaubt später ein einfaches zusammmensetzen der neuen zu den bestehenden Daten.

- [**Cheat Sheet**](https://strftime.org/)
- Python documentation for **strptime**: [string **parse** time, Python 3](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)
- Python documentation for strptime/**strftime**: [string **format** time,Python 3](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

Wandle timestamp_utc noch in ein Datetime-Format um. Könnte auch beim einlesen gemacht werden.

In [70]:
df_data2betested['timestamp_utc'] = pd.to_datetime(df_data2betested['DATUM'], utc=True)

In [71]:
df_data2betested.sort_values('DATUM', ascending=False).head(1)
df_data2betested.dtypes

FK_ZAEHLER                    object
FK_STANDORT                    int64
DATUM                 datetime64[ns]
VELO_IN                      float64
VELO_OUT                     float64
FUSS_IN                      float64
FUSS_OUT                     float64
OST                            int64
NORD                           int64
timestamp_utc    datetime64[ns, UTC]
dtype: object

## Joine die Standortinformationen zu den Messwerten

Wichtig zu wissen ist, dass 
- das Attribut `FK_STANDORT`in den Zählwerten (df_data2betested), der `id1`in den Standortdaten (gdf_zaehler_standorte) entspricht. Ist unschön, aber ist so...
- die räumlichen Standorte über die Zeit ändern können. 

Rename id1 mit 'FK_STANDORT'  für den Join...

In [72]:
gdf_zaehler_standorte = gdf_zaehler_standorte.rename(columns={'id1': 'FK_STANDORT'})
#gdf_zaehler_standorte.columns

In [73]:
df_data2betested = df_data2betested.merge(gdf_zaehler_standorte, on='FK_STANDORT', how="left") #'right', 'inner', 'left'
#df_data2betested.columns
#df_data2betested.sort_values('DATUM', ascending=True)

In der Folge ein paar erste Tests:
 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [74]:
#data2bextended_my.head(2)
df_data2betested.dtypes

FK_ZAEHLER                      object
FK_STANDORT                      int64
DATUM                   datetime64[ns]
VELO_IN                        float64
VELO_OUT                       float64
FUSS_IN                        float64
FUSS_OUT                       float64
OST                              int64
NORD                             int64
timestamp_utc      datetime64[ns, UTC]
id                              object
abkuerzung                      object
bezeichnung                     object
bis                             object
fk_zaehler                      object
korrekturfaktor                float64
objectid                       float64
richtung_in                     object
richtung_out                    object
von                             object
geometry                      geometry
timestamp_von           datetime64[ns]
timestamp_bis           datetime64[ns]
dtype: object

In [75]:
df_data2betested.shape


(786194, 23)

Beschreibe einzelne Attribute

In [76]:
#df_data2betested.describe()


Wie viele Nullwerte gibt es im Datensatz?

In [77]:
df_data2betested.isnull().sum()

FK_ZAEHLER              0
FK_STANDORT             0
DATUM                   0
VELO_IN            322596
VELO_OUT           384220
FUSS_IN            463598
FUSS_OUT           463598
OST                     0
NORD                    0
timestamp_utc           0
id                      0
abkuerzung              0
bezeichnung             0
bis                764510
fk_zaehler              0
korrekturfaktor         0
objectid                0
richtung_in             0
richtung_out        22652
von                     0
geometry                0
timestamp_von           0
timestamp_bis      764510
dtype: int64

## Grafische Auswertungen
### Verwende das Datum als Index

While we did already parse the `datetime` column into the respective datetime type, it currently is just a regular column. 
**To enable quick and convenient queries and aggregations, we need to turn it into the index of the DataFrame**

In [78]:
df_data2betested = df_data2betested.set_index("DATUM")

In [79]:
#df_data2betested.info()
#df_data2betested.index.year.unique()

In [80]:
# first we create the means per week
weekly_means = df_data2betested.resample("W").mean()
# then we generate the weekly means for each quarter
quarterly_means = df_data2betested.resample("Q").mean()
quarterly_median = df_data2betested.resample("Q").median()
# for readability we'll revert the values back to integers
#weekly_means.dropna().astype(int).head(5)
#weekly_means
#quarterly_means
quarterly_median

,FK_STANDORT,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,OST,NORD,korrekturfaktor,objectid
DATUM,,,,,,,,,
2022-03-31,2994.0,3.0,2.0,7.0,6.0,2682756.0,1248451.0,1.1,107.0
2022-06-30,2996.0,5.0,3.0,7.0,6.0,2682689.0,1248451.0,1.0,114.0
2022-09-30,2993.0,6.0,3.0,8.0,7.0,2682731.0,1248435.0,1.0,114.0


### Einfache Visualisierungen zur Plausi

Exploriere die Daten mit Pivottable.JS

Daten zu gross hierfür... Stürzt ab

In [81]:
#from pivottablejs import pivot_ui

#pivot_ui(df_zeitreihe_tb)

### Zeitpunkte und Zeiträume abfragen

A particular powerful feature of the Pandas DataFrame is its indexing capability that also works using time-based entities, such as dates and times. We have already created the index above, so let's put it to use.

### Aggriere Werte nach Zeitausschnitten

Mit den Funktionen zur Zeit kann einfach zwischen Stunden, Tagen, Monaten, etc. gewechselt und aggregiert werden.

Hier z.B. ob es nach dem Zusammenhängen der Jahresbestände flüssige Übergänge gibt oder ob etwas verdächtig aussieht.


In [82]:
#data2betested_my.loc["2017-06-30"]
#df_data2betested[['VELO_IN','VELO_OUT','FUSS_IN','FUSS_OUT']].loc["2022-01-01 00:00":date_today].resample("H").sum()

### Visualisierungen nach Zeitausschnitten

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

Um besser aggregieren zu können hier ein paar Zeitvariabeln, die uns helfen:

In [83]:
df_data2betested['year_str'] = df_data2betested.index.strftime("%Y-%m-%d")
df_data2betested['year_nr'] =df_data2betested.index.year
df_data2betested['month_nr'] = df_data2betested.index.month
df_data2betested['year_month_str'] = df_data2betested.index.strftime("%Y-%m")
df_data2betested['month_name'] = df_data2betested.index.month_name()
df_data2betested['year_month_day_str'] = df_data2betested.index.strftime("%Y-%m-%d")
df_data2betested['day_nr'] = df_data2betested.index.day
df_data2betested['day_name'] = df_data2betested.index.day_name()
df_data2betested['year_month_day_hour_str'] = df_data2betested.index.strftime("%Y-%m-%dT%H:00")
df_data2betested['hour_nr'] = df_data2betested.index.hour

In [123]:
df_data2betested.loc["2022-03-31"].reset_index().head(2).sort_values(["DATUM","FK_ZAEHLER"], ascending=False)

,DATUM,FK_ZAEHLER,FK_STANDORT,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,...,year_month_str,month_name,year_month_day_str,day_nr,day_name,year_month_day_hour_str,hour_nr
1,2022-03-31,Y2H20114444,3012,2.0,1.0,NaN,NaN,...,2022-03,March,2022-03-31,31,Thursday,2022-03-31T00:00,0
0,2022-03-31,U15G3063867,4252,NaN,NaN,1.0,0.0,...,2022-03,March,2022-03-31,31,Thursday,2022-03-31T00:00,0


#### Übersicht zu den Zählstellen

In [122]:
df_zaehlstellen = df_data2betested.loc["2022-01-01":date_today].reset_index()\
    .groupby(['FK_ZAEHLER', 'FK_STANDORT', 'abkuerzung', 'bezeichnung','richtung_in', 'richtung_out']) \
    .agg(sum_VELO_IN =('VELO_IN', 'sum')
         , sum_VELO_OUT =( 'VELO_OUT', 'sum')
         , sum_FUSS_IN =('FUSS_IN', 'sum')
        , sum_FUSS_OUT=('FUSS_OUT', 'sum')
        )
#df_zaehlstellen.sort_values('abkuerzung',ascending=True)

#### Stundensummen ausgewählter Zählstellen
- Hier Fussgänger-Zählstellen am See und am Limmatquai
- Gruppiere nach Stunden

In [187]:
#myAgg = data2betested.loc["2008-11-30":"2021-10-31"]
myAgg = df_data2betested.loc["2022-01-01":date_today].reset_index().query('abkuerzung == "FZS_MYTH" or abkuerzung == "FZS_ARBO" or abkuerzung == "FZS_LIMM" or abkuerzung == "FZS_CASS"')\
    .groupby(['year_month_day_str','FK_ZAEHLER', 'FK_STANDORT', 'abkuerzung', 'bezeichnung','richtung_in', 'richtung_out']) \
    .agg(sum_VELO_IN =('VELO_IN', 'sum')
         , sum_VELO_OUT =( 'VELO_OUT', 'sum')
         , sum_FUSS_IN =('FUSS_IN', 'sum')
        , sum_FUSS_OUT=('FUSS_OUT', 'sum')
        )\
    .assign(
        #Aktualisierungs_Datum_str= lambda x: x.Aktualisierungs_Datum.astype(str),
        total_FUSS = lambda x: x.sum_FUSS_IN +x.sum_FUSS_OUT,
        total_VELO = lambda x: x.sum_VELO_IN +x.sum_VELO_OUT,    
    )\
    .sort_values('sum_FUSS_IN', ascending=False) 

myAgg2=myAgg.reset_index()
myAgg2.head(2)

,year_month_day_str,FK_ZAEHLER,FK_STANDORT,abkuerzung,bezeichnung,richtung_in,richtung_out,sum_VELO_IN,sum_VELO_OUT,sum_FUSS_IN,sum_FUSS_OUT,total_FUSS,total_VELO
0,2022-08-13,YSH18065716,3279,FZS_LIMM,Limmatquai,Rathaus,Seeufer,0.0,0.0,11591.0,9065.0,20656.0,0.0
1,2022-08-13,U15G3063865,1357,FZS_ARBO,Arboretum,Stadtgrenze,Innenstadt,0.0,0.0,9572.0,12382.0,21954.0,0.0


In [188]:
#myAgg2[['year_month_day_hour_str', 'FK_ZAEHLER', 'abkuerzung', 'bezeichnung','total_FUSS','total_VELO']]

##### Zeichne ein einfaches Liniendiagramm der Stundensummen ausgewählter Zählstellen
 - hier ohne Korrekturwerte
 - reminder: Encoding Data Types https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types

In [189]:
myTitle="Stundensummen nach Zählstelle"

highlight = alt.selection(type='single', on='mouseover',fields=['bezeichnung'], nearest=True)

base = alt.Chart(myAgg2[['year_month_day_str', 'FK_ZAEHLER', 'abkuerzung', 'bezeichnung','total_FUSS','total_VELO']], title=myTitle).mark_line(strokeWidth=0.8).encode(
    x=alt.X('year_month_day_str:T', axis=alt.Axis(format='', title='Datum'))
    , y=alt.X('total_FUSS:Q', axis=alt.Axis(title='Anzahl Passanten'))
    , color=alt.Color('bezeichnung:N', legend=alt.Legend(title="Zonen", orient="right"))    
    , tooltip=['year_month_day_str', 'FK_ZAEHLER', 'abkuerzung', 'bezeichnung','total_FUSS','total_VELO']       
)
points = base.mark_circle().encode(
    opacity=alt.value(0.4)
).add_selection(
    highlight
).properties(
    width=750 , height=350
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(0.7), alt.value(2))
).interactive()

lines + points

alt.LayerChart(...)

#### Monats-und Tagessummen aller Zählstellen

In [191]:
#df_data2betested.columns

In [192]:
df_day_medians = df_data2betested.loc["2022-01-01 00:00":date_today].resample("D").median()
df_day_sums = df_data2betested.loc["2022-01-01 00:00":date_today].resample("D").sum()
df_month_medians = df_data2betested.loc["2022-01-01 00:00":date_today].resample("M").median()
df_month_sums = df_data2betested.loc["2022-01-01 00:00":date_today].resample("M").sum()
df_month_sums[['VELO_IN', 'VELO_OUT', 'FUSS_IN', 'FUSS_OUT']]
df_day_sums.head(2)

,FK_STANDORT,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,OST,NORD,korrekturfaktor,objectid,year_nr,month_nr,day_nr,hour_nr
DATUM,,,,,,,,,,,,,
2022-01-01,8637600,5474.0,3446.0,12240.0,11379.0,8498146656,3956282304,3353.3,309984.0,6405696,3168,3168,36432
2022-01-02,8637600,5751.0,3623.0,12286.0,10380.0,8498146656,3956282304,3353.3,309984.0,6405696,3168,6336,36432


In [193]:
#aktuell nimmt die Zeitreihe nicht das laufende Jahr auf.

#months = df_zeitreihe_tb.resample("M").min()
#years = df_zeitreihe_tb.resample("Y").min()

brush = alt.selection(type='interval', encodings=['x'])

upper = alt.Chart(df_month_sums[['VELO_IN', 'VELO_OUT', 'FUSS_IN', 'FUSS_OUT']].reset_index().melt("DATUM")).mark_area(interpolate="basis").encode(
    x = alt.X('DATUM:T', axis=None),
    y = alt.Y('value:Q', axis=None),
    color='variable'
).properties(width=800, height=50).add_selection(brush)

lower = alt.Chart(df_day_sums[['VELO_IN', 'VELO_OUT', 'FUSS_IN', 'FUSS_OUT']].reset_index().melt("DATUM")).mark_line(strokeWidth=1).encode(
    x = alt.X('DATUM:T', scale=alt.Scale(domain=brush)),
    y='value',
    color='variable',
).properties(width=800, height=300)

upper & lower

alt.VConcatChart(...)

**----------------Testing----------- **

**Ich möchte eine Grafik, in der die Jahre farblich eingefärbt werden und der Verlauf jedes Jahres dargestellt wird**

Weitere custimisation, siehe https://altair-viz.github.io/user_guide/customization.html


**--------------- end test ----------------------**

In [185]:
rolling = months_tb[['air_temperature','water_temperature']].rolling(60, center=True, win_type="triang").mean()

chart1 = alt.Chart(rolling.reset_index().melt("timestamp_utc")).mark_line(strokeWidth=1.5, opacity=1).encode(
    x='timestamp_utc', y='value', color='variable',
).properties(width=800, height=400)

# same as the two charts in previous code cell, except more transparent
chart2 = alt.Chart(months_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_line(strokeWidth=1.6, opacity=0.25).encode(
    x='timestamp_utc', y='value', color='variable',
)

chart3 = alt.Chart(years_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_line(interpolate="basis", opacity=0.25).encode(
    x='timestamp_utc', y='value', color='variable',
)

chart1 + chart2 + chart3

NameError: name 'months_tb' is not defined

In [ ]:
# add a column for month names and month number
months_tb['month_number'] = months_tb.index.month
months_tb['month_name'] = months_tb.index.month_name()
months.head(2)

In [ ]:
grp_months_tb = months_tb[['air_temperature','water_temperature', 'month_number', 'month_name']].groupby("month_name").mean()

In [ ]:
alt.Chart(grp_months_tb.reset_index()).mark_bar(width=20).encode(
    x='month_number:O',
    y='air_temperature:Q',
    color='month_name:O'
).properties(width=300, height=300).interactive()

In [ ]:
months_tb = df_zeitreihe_tb.resample("M").median()
years_tb = df_zeitreihe_tb.resample("Y").median()
#months = df_zeitreihe_tb.resample("M").min()
#years = df_zeitreihe_tb.resample("Y").min()

brush = alt.selection(type='interval', encodings=['x'])

upper = alt.Chart(years_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_area(interpolate="basis").encode(
    x = alt.X('timestamp_utc:T', axis=None),
    y = alt.Y('value:Q', axis=None),
    color='variable'
).properties(width=800, height=50).add_selection(brush)

lower = alt.Chart(months_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_line(strokeWidth=1).encode(
    x = alt.X('timestamp_utc:T', scale=alt.Scale(domain=brush)),
    y='value',
    color='variable',
).properties(width=800, height=300)

upper & lower

In [ ]:
#aktuell nimmt die Zeitreihe nicht das laufende Jahr auf.

days_tb = df_zeitreihe_tb.loc["2017-01-01 00:00":"2021-12-31 00:00"].dropna(axis=1).resample("D").median()
months_tb =df_zeitreihe_tb[['air_temperature','water_temperature']].dropna(axis=1).loc["2017-01-01 00:00":"2021-12-31 00:00"].resample("M").median()

years_tb = df_zeitreihe_tb.resample("Y").median()
#months = df_zeitreihe_tb.resample("M").min()
#years = df_zeitreihe_tb.resample("Y").min()

brush = alt.selection(type='interval', encodings=['x'])

upper = alt.Chart(months_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_area(interpolate="basis").encode(
    x = alt.X('timestamp_utc:T', axis=None),
    y = alt.Y('value:Q', axis=None),
    color='variable'
).properties(width=800, height=50).add_selection(brush)

lower = alt.Chart(days_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_line(strokeWidth=1).encode(
    x = alt.X('timestamp_utc:T', scale=alt.Scale(domain=brush)),
    y='value',
    color='variable',
).properties(width=800, height=300)

upper & lower

In [ ]:
#df_zeitreihe_tb[['air_temperature','water_temperature']].dropna(axis=1).loc["2018-12-31 21:50":"2022-01-01 04:30"].resample("M").median()

In [ ]:
months_tb.reset_index().sort_values("timestamp_utc", ascending=False)

## Datenexport

Wenn alle Tests positiv und plausibel sind, kann die neu updateten Datasets als csv exportiert und später veröffentlicht werden.

Checke zuerst kurz, ob im Verlauf der Plausis etwas falsches reingerutscht ist

In [ ]:
df_zeitreihe_tb.shape
#df_zeitreihe_tb.describe()

In [ ]:
df_zeitreihe_my.shape
#df_zeitreihe_my.describe()

### Exportpfade definieren:

#### Dynamisch berechnete Min- und Max-Jahre

In [ ]:
#years.index.year
years = df_zeitreihe_tb.resample("Y").median()
min_year = years.index.year.min()
max_year= years.index.year.max()

print(min_year, max_year, r"\\szh\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_mythenquai_"+str(min_year)+"-"+str(max_year)+".csv")

#### Pfade zusammensetzen

In [ ]:
export_fp_my = r"\\szh\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_mythenquai_"+str(min_year)+"-"+str(max_year)+".csv" #Mythenquai
export_fp_tb= r"\\szh\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_tiefenbrunnen_"+str(min_year)+"-"+str(max_year)+".csv" #Tiefenbrunnen

#### Exportoptionen festlegen

Optionen:
`DataFrame.to_csv(path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', line_terminator=None, chunksize=None, date_format=None, doublequote=True, escapechar=None, decimal='.', errors='strict', storage_options=None)`

### Export ausführen

1. Indexe aufheben. Dabei verliert der indexierte Wert das Iso-Datumsformat
2. Zeitformat bei zuvor indexiertem Datum nochmals ISO-konform definieren: Dies ist nur für utc wichtig, da cet nie verwendet wurde zuvor und weiterhin als String in ISO-Format vorliegt.

##### Mythenquai


In [ ]:
df_my = df_zeitreihe_my.reset_index()
df_my['timestamp_utc'] = df_my.timestamp_utc.apply(datetime.datetime.isoformat)
df_my.head(2)

In [ ]:
df_my.to_csv(export_fp_my
             , sep=','
             , encoding='utf-8-sig'
             ,index=False
                      )

##### Tiefenbrunnen

In [ ]:
df_tb = df_zeitreihe_tb.reset_index()

# Achtung: Reihenfolge spielt hier eine Rolle. 
df_tb['timestamp_utc'] = df_tb.timestamp_utc.apply(datetime.datetime.isoformat)
df_tb.head(2)

In [ ]:
df_tb.to_csv(export_fp_tb
             , sep=','
             , encoding='utf-8-sig'
             ,index=False
            )

### Zeitformatierungen

Alternativ könnte man die `timestamp_cet`beim den Importen noch nicht machen, resp. bei den bestehenden Daten droppen und erst ganz am Schluss alles berechnen.

Aktuell gehe ich so vor:
1. Die neuen Daten importieren --> aus der Datumsangabe --> als utc parsen. Danach cet berechnen
2. Die bisherigen Daten importieren --> die Datumsangaben aber nur als String importieren. Mit utc wird später gerechnet. cet nicht. Am Schluss speichere ich es eigentlich als String, aber merkt man nicht, weil der Export ohne "" bei Strings kommt.

